# Brist1D - Kaggle competition

## Loading and hadling dataset

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv('dataset/train.csv', header=0)
print(df.shape)
df.head(5)

/tmp/ipykernel_46513/722352279.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset/train.csv', header=0)


(177024, 508)


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


## Filtering desired collumns (based on a correlation analises)

In [3]:
def filter_desired_columns(data, include_target: bool = True):
    desired_columns = [
        "id", "p_num", "time",
        "bg-0:00", "bg-0:05", "bg-0:10", "bg-0:15", "bg-0:20", "bg-0:25", "bg-0:30", "bg-0:35", "bg-0:40", "bg-0:45",
        "insulin-0:00", "insulin-0:05", "insulin-0:10", "insulin-0:15", "insulin-0:20",
        "hr-0:00", "hr-0:05", "hr-0:10", "hr-0:15", "hr-0:20", "hr-0:25", "hr-0:30",
        "steps-0:00", "steps-0:05", "steps-0:10", "steps-0:15",
        "cals-0:00", "cals-0:05",
    ]

    if include_target:
        desired_columns.append("bg+1:00")
        
    return data[desired_columns]

df = filter_desired_columns(df)
df.head(5)

,id,p_num,time,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,...,hr-0:20,hr-0:25,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,bg+1:00
0,p01_0,p01,06:10:00,15.1,NaN,NaN,16.2,NaN,NaN,17.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,14.4,NaN,NaN,15.1,NaN,NaN,16.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,13.9,NaN,NaN,14.4,NaN,NaN,15.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,13.8,NaN,NaN,13.9,NaN,NaN,14.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,13.4,NaN,NaN,13.8,NaN,NaN,13.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


## Filling missing values

In [4]:
columns_to_fill = [
    "bg-0:00", "bg-0:05", "bg-0:10", "bg-0:15", "bg-0:20", "bg-0:25", "bg-0:30", "bg-0:35", "bg-0:40", "bg-0:45",
    "insulin-0:00", "insulin-0:05", "insulin-0:10", "insulin-0:15", "insulin-0:20",
    "hr-0:00", "hr-0:05", "hr-0:10", "hr-0:15", "hr-0:20", "hr-0:25", "hr-0:30",
    "steps-0:00", "steps-0:05", "steps-0:10", "steps-0:15",
    "cals-0:00", "cals-0:05",
]

df[columns_to_fill] = df[columns_to_fill].interpolate()

In [5]:
imputer = SimpleImputer()

df[columns_to_fill] = imputer.fit_transform(df[columns_to_fill])

## Converting time into sin and cos pair

### Converting time to minutes

In [6]:
df['minutes_since_start'] = pd.to_datetime(df['time'], format="%H:%M:%S").dt.hour * 60 + pd.to_datetime(df['time'], format="%H:%M:%S").dt.minute
df = df.drop(columns=['time'])

### Converting minutes to sin and cos pair (summing constant 2, to avoid zero values)

In [7]:
df["time_sin"] = np.sin(2 * np.pi / 1440 * df['minutes_since_start']) + 2
df["time_cos"] = np.cos(2 * np.pi / 1440 * df['minutes_since_start']) + 2
df = df.drop(columns=['minutes_since_start'])
df.head(5)

,id,p_num,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,bg-0:35,...,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,bg+1:00,time_sin,time_cos
0,p01_0,p01,15.1,8.246322,8.33178,16.2,8.258647,8.430463,17.3,8.268978,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,13.4,2.999048,1.956381
1,p01_1,p01,14.4,8.246322,8.33178,15.1,8.258647,8.430463,16.2,8.268978,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,12.8,2.994056,1.891133
2,p01_2,p01,13.9,8.246322,8.33178,14.4,8.258647,8.430463,15.1,8.268978,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,15.5,2.984808,1.826352
3,p01_3,p01,13.8,8.246322,8.33178,13.9,8.258647,8.430463,14.4,8.268978,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,14.8,2.971342,1.762314
4,p01_4,p01,13.4,8.246322,8.33178,13.8,8.258647,8.430463,13.9,8.268978,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,12.7,2.953717,1.699294


## Removing ID columns

In [8]:
cols_to_pop = ['id']
identifiers = df[cols_to_pop].copy()
df = df.drop(columns=cols_to_pop)
df.head(5)

,p_num,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,bg-0:35,bg-0:40,...,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,bg+1:00,time_sin,time_cos
0,p01,15.1,8.246322,8.33178,16.2,8.258647,8.430463,17.3,8.268978,8.412561,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,13.4,2.999048,1.956381
1,p01,14.4,8.246322,8.33178,15.1,8.258647,8.430463,16.2,8.268978,8.412561,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,12.8,2.994056,1.891133
2,p01,13.9,8.246322,8.33178,14.4,8.258647,8.430463,15.1,8.268978,8.412561,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,15.5,2.984808,1.826352
3,p01,13.8,8.246322,8.33178,13.9,8.258647,8.430463,14.4,8.268978,8.412561,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,14.8,2.971342,1.762314
4,p01,13.4,8.246322,8.33178,13.8,8.258647,8.430463,13.9,8.268978,8.412561,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,12.7,2.953717,1.699294


In [9]:
df.to_csv("dataset/train_interpolate_circle_causalidade.csv", index=False)

## Spliting train and test dataframes

In [10]:
train = df.sample(frac=0.75, random_state=0)
test = df.drop(train.index)

In [11]:
train_groups = train[["p_num"]].copy() 
print(train_groups.shape)
train.drop(columns=["p_num"], inplace=True)
train.head(5)

(132768, 1)


,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,bg-0:35,bg-0:40,bg-0:45,...,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,bg+1:00,time_sin,time_cos
91143,5.9,5.807317,8.354752,6.2,6.854918,11.098182,4.6,6.721488,11.227333,3.8,...,84.600000,518.000000,573.000000,440.000000,12.00000,8.350000,5.410000,7.6,1.357212,2.766044
35758,8.2,8.200000,8.300000,8.5,8.700000,8.900000,8.5,8.400000,8.500000,8.8,...,76.139606,23.021904,23.077783,23.133661,23.18954,5.600000,5.600000,7.6,2.965926,2.258819
65296,6.8,7.200000,7.900000,8.4,8.700000,8.700000,8.4,8.100000,7.900000,6.5,...,83.500000,8.000000,11.000000,14.000000,72.00000,7.090000,8.630000,6.2,1.123273,1.519011
14652,10.5,10.600000,10.700000,10.8,10.900000,10.900000,11.0,11.200000,11.400000,11.4,...,81.045791,27.270000,27.193333,27.116667,27.04000,1.819452,1.817945,9.0,2.422618,2.906308
54561,6.3,6.300000,6.300000,6.3,6.300000,6.300000,6.2,6.200000,6.200000,6.1,...,52.800000,0.000000,0.000000,0.000000,0.00000,5.720000,5.600000,5.7,2.939693,1.657980


In [12]:
test_groups = test[["p_num"]].copy() 
test_groups = test.drop(columns=["p_num"], inplace=True)
test.head(5)

,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,bg-0:35,bg-0:40,bg-0:45,...,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,bg+1:00,time_sin,time_cos
2,13.9,8.246322,8.33178,14.400000,8.258647,8.430463,15.100000,8.268978,8.412561,16.2000,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,15.5,2.984808,1.826352
10,11.9,8.246322,8.33178,11.400000,8.258647,8.430463,12.700000,8.268978,8.412561,14.8000,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,17.4,2.766044,1.357212
21,5.8,8.246322,8.33178,6.600000,8.258647,8.430463,8.600000,8.268978,8.412561,11.2000,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,4.8,2.152123,1.011638
27,7.8,7.100000,8.33178,7.366667,5.200000,8.430463,7.228571,4.800000,8.412561,7.1250,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,13.1,1.741181,1.034074
30,12.3,7.933333,8.33178,11.066667,8.000000,8.430463,10.271429,7.400000,8.412561,9.7125,...,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,8.3,1.557711,1.103127


In [13]:
x_train = train
y_train = train.pop('bg+1:00')

In [14]:
print(x_train.shape)
x_train.head(5)

(132768, 30)


,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,bg-0:35,bg-0:40,bg-0:45,...,hr-0:25,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,time_sin,time_cos
91143,5.9,5.807317,8.354752,6.2,6.854918,11.098182,4.6,6.721488,11.227333,3.8,...,86.000000,84.600000,518.000000,573.000000,440.000000,12.00000,8.350000,5.410000,1.357212,2.766044
35758,8.2,8.200000,8.300000,8.5,8.700000,8.900000,8.5,8.400000,8.500000,8.8,...,76.143817,76.139606,23.021904,23.077783,23.133661,23.18954,5.600000,5.600000,2.965926,2.258819
65296,6.8,7.200000,7.900000,8.4,8.700000,8.700000,8.4,8.100000,7.900000,6.5,...,86.300000,83.500000,8.000000,11.000000,14.000000,72.00000,7.090000,8.630000,1.123273,1.519011
14652,10.5,10.600000,10.700000,10.8,10.900000,10.900000,11.0,11.200000,11.400000,11.4,...,81.250505,81.045791,27.270000,27.193333,27.116667,27.04000,1.819452,1.817945,2.422618,2.906308
54561,6.3,6.300000,6.300000,6.3,6.300000,6.300000,6.2,6.200000,6.200000,6.1,...,53.000000,52.800000,0.000000,0.000000,0.000000,0.00000,5.720000,5.600000,2.939693,1.657980


In [15]:
print(y_train.shape)
y_train.head(5)

(132768,)


91143    7.6
35758    7.6
65296    6.2
14652    9.0
54561    5.7
Name: bg+1:00, dtype: float64

## Building model

### Imports and GPU config

In [16]:
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import GroupKFold

### Definindo os grupos de features

In [17]:
feature_groups = []

feature_groups.append([
    idx for idx, col in enumerate(train.columns) if col.startswith("bg-")
])

feature_groups.append([
    idx for idx, col in enumerate(train.columns) if col.startswith("insulin-")
])

feature_groups.append([
    idx for idx, col in enumerate(train.columns) if col.startswith("hr-")
])

feature_groups.append([
    idx for idx, col in enumerate(train.columns) if col.startswith("steps-")
])

feature_groups.append([
    idx for idx, col in enumerate(train.columns) if col.startswith("cals-")
])

feature_groups.append([
    idx for idx, col in enumerate(train.columns) if col.startswith("time_")
])

print(feature_groups)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19, 20, 21], [22, 23, 24, 25], [26, 27], [28, 29]]


In [18]:
train_groups = np.array(train_groups)
train_groups = train_groups.ravel()

In [19]:
print("Tipo de groups:", type(train_groups))
print("Shape de groups:", np.shape(train_groups))
print("Valores únicos em groups:", np.unique(train_groups))

Tipo de groups: <class 'numpy.ndarray'>
Shape de groups: (132768,)
Valores únicos em groups: ['p01' 'p02' 'p03' 'p04' 'p05' 'p06' 'p10' 'p11' 'p12']


In [20]:
assert len(train_groups) == len(x_train), "Tamanhos incompatíveis!"

### Preparing test dataframe

In [21]:
x_test = test
y_test = test.pop('bg+1:00')
print(x_test.shape)
x_test.head(5)

(44256, 30)


,bg-0:00,bg-0:05,bg-0:10,bg-0:15,bg-0:20,bg-0:25,bg-0:30,bg-0:35,bg-0:40,bg-0:45,...,hr-0:25,hr-0:30,steps-0:00,steps-0:05,steps-0:10,steps-0:15,cals-0:00,cals-0:05,time_sin,time_cos
2,13.9,8.246322,8.33178,14.400000,8.258647,8.430463,15.100000,8.268978,8.412561,16.2000,...,81.177985,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,2.984808,1.826352
10,11.9,8.246322,8.33178,11.400000,8.258647,8.430463,12.700000,8.268978,8.412561,14.8000,...,81.177985,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,2.766044,1.357212
21,5.8,8.246322,8.33178,6.600000,8.258647,8.430463,8.600000,8.268978,8.412561,11.2000,...,81.177985,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,2.152123,1.011638
27,7.8,7.100000,8.33178,7.366667,5.200000,8.430463,7.228571,4.800000,8.412561,7.1250,...,81.177985,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,1.741181,1.034074
30,12.3,7.933333,8.33178,11.066667,8.000000,8.430463,10.271429,7.400000,8.412561,9.7125,...,81.177985,81.189919,44.927039,43.967174,43.96972,45.359472,8.130857,8.147582,1.557711,1.103127


In [22]:
x_test = x_test.to_numpy()

In [23]:
y_test.head(5)

2     15.5
10    17.4
21     4.8
27    13.1
30     8.3
Name: bg+1:00, dtype: float64

### Model definition

In [24]:
group_kfold = GroupKFold(5)

y_pred = None
i = 0
for idx_train, idx_valid in group_kfold.split(x_train, y_train, groups=train_groups):
    i += 1
    
    x_fold, y_fold = x_train.iloc[idx_train].to_numpy(), y_train.iloc[idx_train].to_numpy()
    x_fold_val, y_fold_val = x_train.iloc[idx_valid].to_numpy(), y_train.iloc[idx_valid].to_numpy()
    
    y_fold = y_fold.reshape(-1, 1)
    y_fold_val = y_fold_val.reshape(-1, 1) 

    model = TabNetRegressor(
        grouped_features=feature_groups, 
    )
    
    model.fit(
        x_fold, y_fold,
        eval_set=[(x_fold_val, y_fold_val)],
        eval_metric=['rmse'],
        batch_size=1024,
    )

    # Prevendo com o modelo atual
    y_pred_fold = model.predict(x_test) / 5

    # Gravando a contribuição do modelo atual
    if y_pred is None:
        y_pred = y_pred_fold
    else:
        y_pred += y_pred_fold

    model.save_model(f"brist1d_v6_1_{i}")


/home/sergiosilva/@pessoal/brist1d/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 12.70478| val_0_rmse: 3.90105 |  0:00:03s
epoch 1  | loss: 4.09144 | val_0_rmse: 2.24542 |  0:00:05s
epoch 2  | loss: 4.00793 | val_0_rmse: 2.17595 |  0:00:08s
epoch 3  | loss: 3.93593 | val_0_rmse: 2.18892 |  0:00:11s
epoch 4  | loss: 3.93051 | val_0_rmse: 2.20399 |  0:00:13s
epoch 5  | loss: 3.88758 | val_0_rmse: 2.16686 |  0:00:16s
epoch 6  | loss: 3.89438 | val_0_rmse: 2.24131 |  0:00:19s
epoch 7  | loss: 3.86263 | val_0_rmse: 2.2033  |  0:00:21s
epoch 8  | loss: 3.82222 | val_0_rmse: 2.15933 |  0:00:24s
epoch 9  | loss: 3.84045 | val_0_rmse: 2.17784 |  0:00:26s
epoch 10 | loss: 3.82534 | val_0_rmse: 2.17319 |  0:00:29s
epoch 11 | loss: 3.8342  | val_0_rmse: 2.25177 |  0:00:32s
epoch 12 | loss: 3.79657 | val_0_rmse: 2.19006 |  0:00:34s
epoch 13 | loss: 3.79088 | val_0_rmse: 2.22593 |  0:00:37s
epoch 14 | loss: 3.76443 | val_0_rmse: 2.22261 |  0:00:40s
epoch 15 | loss: 3.77539 | val_0_rmse: 2.18334 |  0:00:43s
epoch 16 | loss: 3.78091 | val_0_rmse: 2.21483 |  0:00:4

/home/sergiosilva/@pessoal/brist1d/.venv/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at brist1d_v6_1_1.zip


/home/sergiosilva/@pessoal/brist1d/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


...

Successfully saved model at brist1d_v6_1_4.zip


/home/sergiosilva/@pessoal/brist1d/.venv/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 13.92744| val_0_rmse: 3.96629 |  0:00:02s
epoch 1  | loss: 4.17448 | val_0_rmse: 2.50498 |  0:00:05s
epoch 2  | loss: 4.02535 | val_0_rmse: 2.11971 |  0:00:08s
epoch 3  | loss: 3.99753 | val_0_rmse: 2.10215 |  0:00:11s
epoch 4  | loss: 3.90723 | val_0_rmse: 2.0855  |  0:00:13s
epoch 5  | loss: 3.91118 | val_0_rmse: 2.09614 |  0:00:16s
epoch 6  | loss: 3.95837 | val_0_rmse: 2.10076 |  0:00:18s
epoch 7  | loss: 3.86219 | val_0_rmse: 2.09524 |  0:00:21s
epoch 8  | loss: 3.89541 | val_0_rmse: 2.09594 |  0:00:23s
epoch 9  | loss: 3.88275 | val_0_rmse: 2.11378 |  0:00:26s
epoch 10 | loss: 3.85601 | val_0_rmse: 2.12819 |  0:00:28s
epoch 11 | loss: 3.84224 | val_0_rmse: 2.11256 |  0:00:31s
epoch 12 | loss: 3.81825 | val_0_rmse: 2.11482 |  0:00:33s
epoch 13 | loss: 3.82342 | val_0_rmse: 2.10121 |  0:00:36s
epoch 14 | loss: 3.80236 | val_0_rmse: 2.10177 |  0:00:39s

Early stopping occurred at epoch 14 with best_epoch = 4 and best_val_0_rmse = 2.0855


/home/sergiosilva/@pessoal/brist1d/.venv/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at brist1d_v6_1_5.zip


### Calculating error metrics

In [25]:
import tensorflow as tf

mse = tf.keras.losses.MSE(y_test, y_pred).numpy().mean()
mae = tf.keras.losses.MAE(y_test, y_pred).numpy().mean()
rmse = np.sqrt(mse)

2024-11-27 23:52:57.597260: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 23:52:57.608095: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732762377.620132   46513 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732762377.623420   46513 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-27 23:52:57.635773: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [26]:
print(f"RMSE Test: {rmse}")
print(f"MSE Test: {mse}")
print(f"MAE Test: {mae}")

RMSE Test: 1.9549510478973389
MSE Test: 3.821833372116089
MAE Test: 1.450147032737732
